## This script is used to bind the results from all 3 software.
## Written by Yusreen Shah
## Date: May 10th 2023

## The masterfile contains all the results.

In [ ]:
# import the libraries
import numpy as np
import pandas as pd
import re 
import Levenshtein
from Bio import SeqIO
from collections import defaultdict


## This section saves the list of all the queries in a dataframe

In [ ]:
#Read the query and sequences from the .fasta file
fasta_file = "Real_Graph/ERR.fasta" 

# Create empty lists to store the data
queries = []
sequences = []

# Read through the FASTA file and extract query and sequence information
for record in SeqIO.parse(fasta_file, "fasta"):
    queries.append(record.id)
    sequences.append(str(record.seq))

# Create DataFrame from the lists
df = pd.DataFrame({'Query': queries, 'Sequence': sequences})

# Display the DataFrame
print(df)


In [ ]:
# Remove Seq(' and ') from the sequences
df['Sequence'] = df['Sequence'].str.replace('Seq(''', '')
df['Sequence'] = df['Sequence'].str.replace(')', '')
df['Sequence'] = df['Sequence'].str.strip(" \' ")
df['Sequence']=df['Sequence'].str.rstrip()

In [ ]:
df=df.sort_values("Query")
df=df.reset_index(drop=True)
df

In [ ]:
df.to_csv("testSequences.tsv",sep="\t")

## This section is used to compare the results from Bandage and the actual list of queries.

In [ ]:
# Create a dataframe for Bandage Combined1
Bandage_Combined1= pd.read_csv('Real_Results/realBandage.tsv', sep='\t')
Bandage_Combined1

In [ ]:
# Perform a left merge on 'Query' column to compare sequences
merged_df_test = pd.merge(df,Bandage_Combined1, on='Query', how='right')

merged_df_test

# Calculate the difference between 'Sequence_x' and 'Sequence_y'
merged_df_test['Distance'] = Levenshtein.distance(merged_df_test['Sequence_x'],merged_df_test['Sequence_y'])
# Create a new dataframe with unique values in the 'Query' column
df_unique = df.drop_duplicates(subset='Query', keep='first')


#merged_df_test.to_csv("CAMIH1_Results/TestDistance.tsv",sep="\t")
# Print the new dataframe
df_unique
df_unique.to_csv("111_GraphResults/TestDistance.tsv",sep="\t")

In [ ]:
df_unique

In [ ]:
#len(df.index)
Bandage_Combined1.dtypes

In [ ]:
Bandage_Combined_Query_Path =Bandage_Combined1[['Path','Query','Sequence']]

## Extract the start and end position from the path in Bandage

In [ ]:
# Extract the start position
pattern_path_start = r'\((.*?)\)' 

In [ ]:
#Extract the node position
pattern_path_node = r'\b(\d+)\s*[+-]'

In [ ]:
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Path'].apply(lambda x: re.findall(pattern_path_node, x))

In [ ]:
# Remove the braces and convert the values to integers
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].apply(lambda x: [int(value) for value in x])

Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].astype(str)
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].str.replace('[', '').str.replace(']', '')


In [ ]:

Bandage_Combined_Query_Path.to_csv("test4.tsv",sep="\t")

In [ ]:
Bandage_Combined_Query_Path['Start']=Bandage_Combined_Query_Path['Path'].str.extract(pattern_path_start, expand=False)
Bandage_Combined_Query_Path

In [ ]:
# Extract the end position
Bandage_Combined_Query_Path['End']= Bandage_Combined_Query_Path["Path"].str.split().str[-1]


In [ ]:
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('()')
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('+')
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('-')
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].astype(int)
Bandage_Combined_Query_Path

In [ ]:
# Reorder the columns in Bandage_Combined_Query_Path
Bandage_Combined_Query_Path=Bandage_Combined_Query_Path[['Query','Path','Start','Extracted_Path','End','Sequence']]

# Convert integer columns to strings
Bandage_Combined_Query_Path['Path'] = Bandage_Combined_Query_Path['Path'].astype(str)
Bandage_Combined_Query_Path['End'] = Bandage_Combined_Query_Path['End'].astype(str)
Bandage_Combined_Query_Path['Start'] = Bandage_Combined_Query_Path['Start'].astype(str)
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].astype(str)
Bandage_Combined_Query_Path['Sequence'] = Bandage_Combined_Query_Path['Sequence'].astype(str)

# Bandage_Combined1['Query'].value_counts() 
# Merge the rows that have the same query
# group the dataframe by the 'Name' column and aggregate the data for each group
merge_Bandage_df = Bandage_Combined_Query_Path.groupby('Query').agg({'Path': ' , '.join, 'Start': ', '.join, 'End': ', '.join, 'Extracted_Path': ','.join, 'Sequence': ' '.join}).reset_index()
merge_Bandage_df


In [ ]:
# Add Braces to the values that now have ','
def add_braces(value):
    if ',' in value:
        return '{' + value + '}'
    else:
        return value
#merge_Bandage_df['Path'] = merge_Bandage_df['Path'].apply(add_braces)
#merge_Bandage_df['Start'] = merge_Bandage_df['Start'].apply(add_braces)
#merge_Bandage_df['End'] = merge_Bandage_df['End'].apply(add_braces)
#merge_Bandage_df['Sequence'] = merge_Bandage_df['Sequence'].apply(add_braces)

In [ ]:
# Store the dataframe containing the Bandage results to a .tsv file
merge_Bandage_df.to_csv("SequenceBandage.tsv", sep="\t")

In [ ]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsBandage=pd.merge(df,merge_Bandage_df, on='Query',how="outer")

## Compile the Queries and the results from Bandage in one dataframe

In [ ]:
df_QueryAndResultsBandage

In [ ]:
#Print the columns from the dataframe
df_QueryAndResultsBandage.columns

In [ ]:
#Rename the columns from the merging of Bandage and Query list
df_QueryAndResultsBandage.rename(columns = {'Sequence_x':'Sequence'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Sequence_y':'Sequence_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Length':'Length_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Path':'Path_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Start':'Start_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'End':'End_Bandage'}, inplace = True)

## Save the Queries and Results from Bandage in one .tsv file

In [ ]:
df_QueryAndResultsBandage.to_csv("CompiledQueriesBandage.tsv",sep="\t")
df_QueryAndResultsBandage

## This section is used to compare the results from SPAligner and the actual list of queries.

In [ ]:
#Create a dataframe for SPAligner Combined1
SPAligner_Combined1=pd.read_csv('Real_Results/realSPAligner.tsv', sep='\t',
names=["Query","Start position of alignment on sequence","End position of alignment on sequence","Start position of alignment on the first edge of the path",
"End position of alignment on the last edge of the path","Length","Path of alignment","Sequence Length",
"Sequence"])
SPAligner_Combined1

In [ ]:
#get the columns from SPAligner
SPAligner_Combined1.columns

In [ ]:
#format the Query column from  SPAligner_Combined1 so that the Query column is the same as the one from Bandage
pattern = r'\[.*?\]'
def format_query(x):
    return re.sub(pattern,"", x).rstrip()
    


SPAligner_Combined1['Query'] = SPAligner_Combined1['Query'].map(format_query)


In [ ]:
#Get the query and sequence column from SPAligner_Combined1
SPAligner_Combined_Query_Path=SPAligner_Combined1[['Query','Sequence']]


In [ ]:
#Get the query and sequence column from SPAligner_Combined1
SPAligner_Combined_Query_Path=SPAligner_Combined1[['Query','Start position of alignment on the first edge of the path', 'End position of alignment on the last edge of the path', 'Sequence Length',
'Path of alignment','Sequence']]
SPAligner_Combined_Query_Path

In [ ]:
#Rename the columns in SPAligner
SPAligner_Combined_Query_Path.rename(columns = {'Start position of alignment on the first edge of the path':'Start_SPAligner'}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'End position of alignment on the last edge of the path':'End_SPAligner'}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Sequence Length':"Length_SPAligner"}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Path of alignment':"Path_SPAligner"}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Sequence':"Sequence_SPAligner"}, inplace = True)

SPAligner_Combined_Query_Path

In [ ]:
SPAligner_Combined_Query_Path

## Compile the Queries and the results from SPAligner in one dataframe

In [ ]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsSPAligner=pd.merge(df,SPAligner_Combined_Query_Path, on='Query',how='right')

df_QueryAndResultsSPAligner

In [ ]:
df_QueryAndResultsSPAligner.to_csv("TestCompiledQueriesSPAligner.tsv",sep="\t")

In [ ]:
#Rename the columns from SPAligner and Results from the Query list
df_QueryAndResultsSPAligner.rename(columns = {'Sequence_x':'Sequence'}, inplace = True)
df_QueryAndResultsSPAligner.rename(columns = {'Sequence_y':'Sequence_SPAligner'}, inplace = True)

In [ ]:
#df_QueryAndResultsSPAligner['Start_SPAligner']=df_QueryAndResultsSPAligner['Start_SPAligner']+1

#df_QueryAndResultsSPAligner

In [ ]:
df_QueryAndResultsSPAligner.to_csv("TestCompiledQueriesSPAligner.tsv",sep="\t")

## This section is used to compare the results from GraphAligner and the actual list of queries.

In [ ]:
#Create a dataframe for GraphAligner Combined1
GraphAligner_Combined1=pd.read_csv('Real_Results/RealGraphAligner.tsv', sep='\t', names=["Query", "Query Length", "Query Start", 
                                        "Query End","Strand Relative Length","Path Matching","Path Length",
                                         "Start Position on Path","End Position on Path","Number of residues Matches",
                                         "Alignment Back Length","Mapping Quality","Column 1"])
GraphAligner_Combined1

In [ ]:
#Format the Query column from  GraphAligner_Combined1 so that the Query column is the same as the one from Bandage
pattern = r'\[.*?\]'
def format_query(x):
    return re.sub(pattern,"", x).rstrip()
    


GraphAligner_Combined1['Query'] = GraphAligner_Combined1['Query'].map(format_query)

In [ ]:
# Remove < and > from the Path
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('>', " ")
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('<', " ")

## This section drops the rows with coverages < 50 from GraphAligner_Combined1

In [ ]:
# Calculate the coverage for GraphAligner
GraphAligner_Combined1["Coverage"]=GraphAligner_Combined1["Alignment Back Length"]/GraphAligner_Combined1["Query Length"]*100

In [ ]:
GraphAligner_Combined1

In [ ]:
#Drop the coverages that are < 50 from GraphAligner 
GraphAligner_Combined1.drop(GraphAligner_Combined1[GraphAligner_Combined1['Coverage'] < 50].index, inplace = True)


In [ ]:
GraphAligner_Combined1['Start Position on Path']=GraphAligner_Combined1['Start Position on Path']+1
GraphAligner_Combined1['End Position on Path']=GraphAligner_Combined1['End Position on Path']
GraphAligner_Combined1

In [ ]:
#Concatenate the values from Path_Matching
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].apply(lambda x: ', '.join(x.split()))


In [ ]:
#Get the specific columns from GraphAligner
GraphAligner_Combined_Query_Path=GraphAligner_Combined1[['Query','Path Matching', 'Start Position on Path', 
'End Position on Path']]
# Convert the integer to string
GraphAligner_Combined_Query_Path['Query']=GraphAligner_Combined_Query_Path['Query'].astype(str)
GraphAligner_Combined_Query_Path['Path Matching']=GraphAligner_Combined_Query_Path['Path Matching'].astype(str)
GraphAligner_Combined_Query_Path['Start Position on Path']=GraphAligner_Combined_Query_Path['Start Position on Path'].astype(str)
GraphAligner_Combined_Query_Path['End Position on Path']=GraphAligner_Combined_Query_Path['End Position on Path'].astype(str)



In [ ]:
#Use merge operation so that we have the sequences for both softwares for successful queries
merged_df_Graph = GraphAligner_Combined_Query_Path.groupby('Query').agg({'Path Matching':' '.join, 'Start Position on Path': ', '.join,'End Position on Path': ', '.join}).reset_index()
merged_df_Graph

In [ ]:
merged_df_Graph.rename(columns = {'Path Matching':'Path_GraphAligner'}, inplace = True)
merged_df_Graph.rename(columns = {'Start Position on Path':'Start_GraphAligner'}, inplace = True)
merged_df_Graph.rename(columns = {'End Position on Path':'End_GraphAligner'}, inplace = True)

In [ ]:
merged_df_Graph

## Compile the Queries and the results from GraphAligner in one dataframe

In [ ]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsGraphAligner=pd.merge(df,merged_df_Graph, on='Query',how="left")
df_QueryAndResultsGraphAligner

## Compile all the results in one master Dataframe

In [ ]:
merged_df_1 = pd.merge(df_QueryAndResultsBandage, df_QueryAndResultsSPAligner, on=['Query', 'Sequence'], how='outer')
merged_df_master = pd.merge(merged_df_1, df_QueryAndResultsGraphAligner, on=['Query', 'Sequence'], how='outer')
#merged_df_master['Start_SPAligner']=merged_df_master['Start_SPAligner'].astype(float).astype(int)
merged_df_master

In [ ]:
#Drop some columns

merged_df_master.drop(['Sequence_y', 'Length_x','Length_y','Sequence'], axis=1)

#Rename Sequence_x
merged_df_master.rename(columns = {'Sequence_x':'Sequence_Query'}, inplace = True)



In [ ]:
def compare_paths(Start_Bandage, End_Bandage,Start_GraphAligner,End_GraphAligner):
    bandage_path_start = Start_Bandage.split(',')
    bandage_path_end = End_Bandage.split(',')
    #str(Start_GraphAligner) in str(bandage_path_start) and  str(End_GraphAligner) in  str(bandage_path_end):
    
    
    if set(str(bandage_path_start)) == set([str(Start_GraphAligner)]) and set(str(bandage_path_end) )== set([str(End_GraphAligner)]):
        return 'Similar'
    elif str(Start_GraphAligner) in str(bandage_path_start) and  str(End_GraphAligner) in  str(bandage_path_end):
        return 'Full'
    else:
        return 'Different'

In [ ]:
merged_df_master['BandageVSGraphAligner'] = merged_df_master.apply(lambda row: compare_paths(row['Start_Bandage'], row['End_Bandage'],row['Start_GraphAligner'],row['End_GraphAligner']), axis=1)

merged_df_master

In [ ]:
#Compare the Start, and end path from Bandage and SPAligner
merged_df_master['Start_SPAligner']=merged_df_master['Start_SPAligner'].astype(str)
merged_df_master['End_SPAligner']=merged_df_master['End_SPAligner'].astype(str)
merged_df_master['BandageVSSPAligner']=(merged_df_master['Start_Bandage'] == merged_df_master['Start_SPAligner']) & (merged_df_master['End_Bandage'] == merged_df_master['End_SPAligner'])



merged_df_master.dtypes

In [ ]:
#Compare the Start, and end path from GraphAligner and SPAligner
merged_df_master['SPAlignerVSGraphAligner']=(merged_df_master['Start_SPAligner'] == merged_df_master['Start_GraphAligner']) & (merged_df_master['End_SPAligner'] == merged_df_master['End_GraphAligner'])

In [ ]:
#Drop some columns

merged_df_master=merged_df_master.drop(['Sequence_y', 'Length_x','Length_y','Sequence'], axis=1)

## Save the Queries and  all the Results  in one .tsv file


In [ ]:
merged_df_master.to_csv("Real_Results/QueriesAndResultsMasterList.tsv",sep='\t')

## This section

merged_df_master['Sequence_Bandage']=merged_df_master['Sequence_Bandage'].astype(str)

In [ ]:
def extract_closest_string(df):
    df['Extracted string'] = ''

    for index, row in df.iterrows():
        sequences = row['Sequence_Bandage']
        query = row['Sequence']
        closest_distance = float('inf')
        closest_string = ''

        if sequences:
            sequences = sequences.split(' ')
            for sequence in sequences:
                distance = Levenshtein.distance(sequence, query)
                if Levenshtein.distance(sequence, query) < closest_distance:
                    closest_distance = distance
                    closest_string = sequence

        df.at[index, 'Extracted string'] = closest_string


    return df

## This section saves the Nodes and Sequences from the gfa file

In [ ]:
import gfapy

# Open the GFA file
file_path = "graph1.gfa"
gfa = gfapy.Gfa.from_file(file_path)

num_segments = len(gfa.segments)

In [ ]:
data_graph = []

In [ ]:
# Store the name and sequence for each node from the graph
for segment in gfa.segments:
    data_graph.append({"Name": segment.name, "Sequence": segment.sequence})

# Convert the list to a DataFrame
df_graph = pd.DataFrame(data_graph)

# Print the DataFrame
print(df_graph)

In [ ]:
#Count the number of duplicates
#len(df_graph['Name'])-len(df_graph['Name'].drop_duplicates())

In [ ]:
num_segments

In [ ]:
#Save only one copy of each row to the dataframe
df_graph=df_graph.drop_duplicates()

In [ ]:
df_graph

In [ ]:
df_graph['Name']=df_graph['Name'].astype(str)


## Get the sequences unique to GraphAligner
## Get the dataframe containing the path from GraphAligner


In [ ]:
df_GraphAligner_Paths= pd.read_csv('GraphAlignerNotEqual.tsv', sep='\t')
#GraphAlignerNotEqual.tsv is from another script:GraphAlignerBandageScript.ipynb

In [ ]:
df_GraphAligner_Paths

In [ ]:
df_GraphAligner_Paths
df_GraphAligner_Paths.sort_index(inplace=True)
df_GraphAligner_Paths['Path Matching']=df_GraphAligner_Paths['Path Matching'].astype(str)

In [ ]:
# Function to process the Path values
def process_path(path):
    path = path.replace('<', ' ').replace('>', ' ')
    path = ' '.join(path.split('<'))
    return path

In [ ]:
df_GraphAligner_Paths['Path Matching']=df_GraphAligner_Paths['Path Matching'].apply(process_path).str.strip()


In [ ]:
df_GraphAligner_Paths
# Create a new column 'Combined_Sequence' that concatenates sequences if Path has two values


In [ ]:
df_GraphAligner_Paths[['Path1', 'Path2']] = df_GraphAligner_Paths['Path Matching'].str.split(' ', expand=True)

In [ ]:
df_GraphAligner_Paths

In [ ]:
# Get the rows where Path 2 is not None
rows_not_none = df_GraphAligner_Paths[df_GraphAligner_Paths['Path2'].notna()]

# Print the resulting dataframe
rows_not_none

In [ ]:
#Get the sequence for Path 1 and Path 2
# Merge df1 and df2 based on matching values in Column1 and Name

merged_df= pd.merge(df_GraphAligner_Paths, df_graph, left_on='Path1', right_on='Name')
merged_df = pd.merge(merged_df, df_graph, left_on='Path2', right_on='Name')

# Concatenate the sequences
merged_df

In [ ]:
extracted_sequences_Path1_df = extract_sequences_Path1(merged_df_Path1)
extracted_sequences_Path1_df
#Save the dataframe to a pdf
extracted_sequences_Path1_df.to_csv("Path1_TestSequences.tsv", sep="\t")